In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#__author__ = 'Prakhar MISRA'
# Created 8/18/2019
#Last edit 8/18/2010

'''# Purpose: Undo seasona s, r and use REAS season emissions'''

#Contents:
# 1. cosnider 10km, 20km, 30km radius neughborhood of the central pixel
# 2. count pixels greater than a threshold.
# 3. save this information at daily level and the annual level

# NOTE
# only plotfrp_annual, plotfrp_annual_bymonth works, linkis for aother functins are broken


import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
sns.set_style("whitegrid")
from datetime import timedelta, date
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt
import os

currdir = os.getcwd()

In [5]:
# filename whcih needs to be undone
dfAY_all_path  = os.path.join(currdir, "my_bayesian",  "HBM_city_tier_20180116","report20180212_IDW2_20190814", "meteorology_model", 'allAY20180212IDW2setclean_meteor.csv')

# final undoen save path
# the prevoous file titled allAY20180212IDW2setclean_meteor_sr2 refers to incorrect sr
dfAY_all_path_undone_sr = os.path.join(currdir, "my_bayesian",  "HBM_city_tier_20180116","report20180212_IDW2_20190814", "meteorology_model", 'allAY20180212IDW2setclean_meteor_sr2.csv')
dfAY_all_path_undone_sr20012011 = os.path.join(currdir, "my_bayesian",  "HBM_city_tier_20180116","report20180212_IDW2_20190814", "meteorology_model", 'allAY20180212IDW2setclean20012011_meteor_sr2.csv')

# city list
citylist = ['Agra', 'Ahmedabad', 'Allahabad', 'Chennai', 'Kanpur', 'Lucknow', 'Ludhiana', 'Patna', 'Raipur', 'Hyderabad', 'Jaipur', 'Bangalore', 'Kolkata', 'NewDelhi', 'Mumbai']
#citylist = ["Amritsar"]


In [6]:
# open the df
dfAY_all = pd.read_csv(dfAY_all_path, header = 0)

dfAY_all['date'] = pd.to_datetime(dfAY_all['date'], format="%m/%d/%Y")
dfAY_all["year"] = dfAY_all["date"].dt.year
# first get A_LU without any s, r correction


In [7]:

# get the maximum oirignal values
df_originalmax = dfAY_all.groupby(["city_coord", "year"]).max()[["AR", "AC"]]
df_originalmax.columns = ["ARu", "ACu"]

# also rename the seasonally corrected columns from AR, AC to ARo, ACo

# now make two index
dfAY_all.index = pd.MultiIndex.from_arrays(dfAY_all[['city_coord', 'year']].values.T, names=['idx1', 'idx2'])
#dfAY_all.index = pd.MultiIndex.from_arrays(dfAY_all[['city_coord', 'year']].values.T, names=['idx1', 'idx2'])



In [8]:
# need to give name to index level
dfAY_all.index.levels[0].name = "idx1"
dfAY_all.index.levels[1].name = "idx2"
df_originalmax.index.levels[0].name = "idx1"
df_originalmax.index.levels[1].name = "idx2"


# now merge
df_merge = pd.merge(dfAY_all, df_originalmax, how='inner', on=None, left_on=None, right_on=None,
         left_index=True, right_index=True, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)



In [10]:
# assume new seasonal emission
df_merge["s"] = 0.8
df_merge["r"] = 0.8
df_merge.loc[df_merge.month.isin([2,3]),"s"] = 0.9
df_merge.loc[df_merge.month.isin([4,5,6]),"s"] = 0.85
df_merge.loc[df_merge.month.isin([7,8,9]),"s"] = 0.75
df_merge.loc[df_merge.month.isin([10]),"s"] = 0.8
df_merge.loc[df_merge.month.isin([11,12,1]),"s"] = 1

# get the new A_LC
df_merge["AR"] = df_merge["ARu"]*df_merge["s"]
df_merge["AC"] = df_merge["ACu"]*df_merge["s"]

In [11]:
# save the csv
df_merge.to_csv(dfAY_all_path_undone_sr)

In [12]:
# save the csv
df_merge.loc[df_merge["year"].isin([2001,2011]),:].to_csv(dfAY_all_path_undone_sr20012011)

R  month  AvgHumidity      beta            AR  \
idx1       idx2                                                          
Agra       2001  47.752187      1         0.64  1.272727  20792.459200   
           2001  36.265523      2         0.55  1.181818  16218.118179   
           2001  24.971959      3         0.45  1.181818  14970.570624   
           2001  38.929874      4         0.35  1.090909  10604.154192   
           2001  50.031110      5         0.34  1.090909  10604.154192   
           2001  46.093650      6         0.46  1.181818   9425.914834   
           2001  42.063698      7         0.72  1.272727   7797.172200   
           2001  42.063698      8         0.77  1.272727   6237.737759   
           2001  47.693265      9         0.67  1.272727   6237.737759   
           2001  51.812673     10         0.54  1.181818  11643.777152   
           2001  53.845396     11         0.56  1.181818  20792.459200   
           2001  56.109313     12         0.64  1.272727  20792.459200   
           2011  55.526164      1         0.64  1.272727  27831.368500   
           2011  46.754165      2         0.55  1.181818  21708.467433   
           2011  35.730796      3         0.45  1.181818  20038.585320   
           2011  44.346466      4         0.35  1.090909  14193.997935   
           2011  34.464408      5         0.34  1.090909  14193.997935   
           2011  52.977210      6         0.46  1.181818  12616.887059   
           2011  71.871016      7         0.72  1.272727  10436.763188   
           2011  78.571429      8         0.77  1.272727   8349.410550   
           2011  25.719011      9         0.67  1.272727   8349.410550   
           2011  72.678720     10         0.54  1.181818  15585.566360   
           2011  71.931806     11         0.56  1.181818  27831.368500   
           2011  72.209963     12         0.64  1.272727  27831.368500   
Agra1_2    2001  52.331004      1         0.64  1.272727  20238.796460   
           2001  27.837330      2         0.55  1.181818  15786.261243   
           2001  21.446219      3         0.45  1.181818  14571.933453   
           2001  42.639514      4         0.35  1.090909  10321.786198   
           2001  29.384570      5         0.34  1.090909  10321.786198   
           2001  51.987429      6         0.46  1.181818   9174.921063   
...                    ...    ...          ...       ...           ...   
Raipur2_3  2011  18.262330      6         0.61  1.272727   5711.356196   
           2011  45.144622      7         0.82  1.443540   4724.467441   
           2011  24.953368      9         0.79  1.272727   3779.573953   
           2011  35.170805     10         0.68  1.272727   7055.204712   
           2011  36.548155     11         0.62  1.272727  12598.579840   
           2011  31.040925     12         0.57  1.181818  12598.579840   
Raipur3_10 2001  38.116224      1         0.54  1.181818   3872.893157   
           2001  30.730441      2         0.48  1.181818   3020.856663   
           2001  45.684567      3         0.40  1.181818   2788.483072   
           2001  33.589608      4         0.37  1.090909   1975.175510   
           2001  28.289585      5         0.35  1.090909   1975.175510   
           2001  18.168191      6         0.61  1.272727   1755.711564   
           2001  26.883826      7         0.82  1.443540   1452.334934   
           2001  19.609352      8         0.83  1.464208   1161.867947   
           2001  34.558711      9         0.79  1.272727   1161.867947   
           2001  22.645823     10         0.68  1.272727   2168.820168   
           2001  30.037727     11         0.62  1.272727   3872.893157   
           2001  24.662913     12         0.57  1.181818   3872.893157   
           2011  34.882856      1         0.54  1.181818  11961.293130   
           2011  39.458994      2         0.48  1.181818   9329.808645   
           2011  31.168354      3         0.40  1.181818   8612.131057   
           2011  43.109782   